In [15]:
from jira import JIRA
from dotenv import load_dotenv
import os
import openai
load_dotenv()
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import langchain
# langchain.verbose=True
# langchain.debug=True

JIRA_USERNAME=os.getenv("JIRA_USERNAME")
JIRA_INSTANCE_URL=os.getenv("JIRA_INSTANCE_URL")
JIRA_API_TOKEN=os.getenv("JIRA_API_TOKEN")

jira = JIRA(server=JIRA_INSTANCE_URL,basic_auth=(JIRA_USERNAME,JIRA_API_TOKEN))

In [17]:
def create_corpus():
    corpus = []

    projects = jira.projects()
    for project in projects:

        project_data = {
            "project_key": project.key,
            "project_name": project.name,
            "issues": []
        }
        
        issues = jira.search_issues(f'project={project.key}', maxResults=100)
        for issue in issues:
            issue_data = {
                "issue_key": issue.key,
                "issue_summary": issue.fields.summary,
                "issue_type": issue.fields.issuetype.name,
                "issue_status": issue.fields.status.name,
                "comments": []
            }

            comments = jira.comments(issue)
            for comment in comments:
                comment_data = {
                    "comment_id": comment.id,
                    "comment_author": comment.author.displayName,
                    "comment_body": comment.body
                }
                issue_data["comments"].append(comment_data)
            
            project_data["issues"].append(issue_data)
        
        corpus.append(project_data)
    
    return corpus

def flatten_corpus(corpus):
    texts = ''
    for project in corpus:
        for issue in project['issues']:
            texts+=f"Issue/Ticket:  Project:{project['project_name']}, Issue Key: {issue['issue_key']}, Summary: {issue['issue_summary']}, Type: {issue['issue_type']}, Status: {issue['issue_status']} \n  "
            for comment in issue['comments']:
                texts+=f"Comment: Comment ID: {comment['comment_id']}, Author: {comment['comment_author']}, Body: {comment['comment_body']}, Issue Key: {issue['issue_key']} \n  "
    return texts

def get_vector_store(text, embeddings):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    knowledgeBase = FAISS.from_texts(chunks, embeddings)
    return knowledgeBase

In [32]:
corpus = create_corpus()

for project in corpus[:1]:
    print(f"Project Key: {project['project_key']}, Project Name: {project['project_name']}")
    for issue in project['issues'][:1]:
        print(f"Issue Key: {issue['issue_key']}, Summary: {issue['issue_summary']}")
        for comment in issue['comments'][:1]:
            print(f"Comment ID: {comment['comment_id']}, Author: {comment['comment_author']}, Body: {comment['comment_body']}")

Project Key: SHAED, Project Name: SHAED
Issue Key: SHAED-3611, Summary: Dev | PNI | Admin/Sales/Sale support | Post Order | Deal Jacket | Download button is not working.
Comment ID: 17819, Author: Hassaan Qureshi, Body: → Verified on [dev-prithchards.shaed.ai|http://dev-prithchards.shaed.ai]

!Screenshare - 2024-04-02 7_42_58 PM.mp4|width=1920,height=1080,alt="Screenshare - 2024-04-02 7_42_58 PM.mp4"!


In [34]:
text = flatten_corpus(corpus)
embeddings = OpenAIEmbeddings()
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)
vector_store = get_vector_store(text, embeddings)

In [44]:
prompt = ChatPromptTemplate.from_template("""Answer the questions based on the context provided. When a user refers to an issue or a ticket they will be labelled in context by Issue/Ticket:. When a user refers to a comment 
they will be labelled by Comment:. The comments will also have an issue key which will connect them to a ticket/issue.Format the output According to the Output Format given below.A new line in format will be referenced by
(newline).

Output Format In Case of An Issue/Ticket:
Project Key: (project key)(newline)
Issue Key: (issue key)(newline)
Issue Summary: (issue summary)(newline)
Issue Type: (issue type)(newline)
Issue Status: (issue status)(newline)

Output Format In Case of A Comment:
Issue Key: (issue key)(newline)
Comment ID: (comment id)(newline)
Author: (author)(newline)
Body: (body)(newline)
                                                                                                                                                                                                                                                                                                                                                                                       

Context:
{context}

                                        
                             
Based on the context above,
Question: {input}

""")

In [45]:
document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vector_store.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [46]:
def query_assistant(input):
    response = retrieval_chain.invoke({"input": input})
    return response["answer"]

In [47]:
query_assistant('can you tell me some comments where a /mp4 file was referenced?')

[chain/start] [1:chain:retrieval_chain] Entering Chain run with input:
{
  "input": "can you tell me some comments where a /mp4 file was referenced?"
}
[chain/start] [1:chain:retrieval_chain > 2:chain:RunnableAssign<context>] Entering Chain run with input:
{
  "input": "can you tell me some comments where a /mp4 file was referenced?"
}
[chain/start] [1:chain:retrieval_chain > 2:chain:RunnableAssign<context> > 3:chain:RunnableParallel<context>] Entering Chain run with input:
{
  "input": "can you tell me some comments where a /mp4 file was referenced?"
}
[chain/start] [1:chain:retrieval_chain > 2:chain:RunnableAssign<context> > 3:chain:RunnableParallel<context> > 4:chain:retrieve_documents] Entering Chain run with input:
{
  "input": "can you tell me some comments where a /mp4 file was referenced?"
}
[chain/start] [1:chain:retrieval_chain > 2:chain:RunnableAssign<context> > 3:chain:RunnableParallel<context> > 4:chain:retrieve_documents > 5:chain:RunnableLambda] Entering Chain run with i

'Comment: Comment ID: 17657, Author: Hassaan Qureshi, Body: → Here is the video recording for the PDF file issue. [~accountid:712020:929a402d-61a4-4ee2-9b9b-558d14102d77] \n!Screenshare - 2024-03-27 4_23_48 PM.mp4|width=1920,height=1080,alt="Screenshare - 2024-03-27 4_23_48 PM.mp4"!, Issue Key: SHAED-3563(newline)\nComment: Comment ID: 17686, Author: Zaeem Khaliq, Body: [~accountid:642ae5b67e859baa54f552be] Try now, I am able to download it now.\n!bandicam 2024-03-28 07-36-48-336.mp4|width=1912,height=996,alt="bandicam 2024-03-28 07-36-48-336.mp4"!, Issue Key: SHAED-3543(newline)\nComment: Comment ID: 17632, Author: Hassaan Qureshi, Body: → Download button is not working. There should be a loader for the Download button if the file is going to take some time to download.\n!Screenshare - 2024-03-27 12_56_37 PM.mp4|width=1920,height=1080,alt="Screenshare - 2024-03-27 12_56_37 PM.mp4"!, Issue Key: SHAED-3536(newline)\nComment: Comment ID: 17725, Author: Muhammad Jamali, Body: [~accountid: